In [1]:
#Libraries
from pyspark import SparkContext # spark
from pyspark.streaming import StreamingContext # spark streaming
import sys
import pymongo
from pymongo import MongoClient
from pprint import pprint #for printing Mongodb output in the queries
import json
import datetime as dt
import pandas as pd

In [2]:
client = MongoClient () #defining the Mongodb client.
result = client.drop_database('as2TaskB') #ensure that the as2TaskB database 
                                          #doesn't already exist

db = client.as2TaskB #defining the db

fireCollection = db.fire #define a new collection for fire data. 
                         #This will store fire data plus the 
                         #the embedded climate data associated 
                         #with each fire

climateCollection = db.climate #define a new collection 
                               #for the climate data

result = fireCollection.drop() #Ensure that the fire collection 
                               #does not already exist in Mongo db

result = climateCollection.drop() #Ensure that the fire collection 
                                  #does not already exist in Mongo db

In [3]:
def sendNewPartition(iter):
    connection = MongoClient()
    db = connection.get_database('as2TaskB')
    climateCollection = db.climate
    fireCollection = db.fire
    
    for record in iter:
        #print(record)
        datestamp = record[0]
        #climateRec = record[1]
        fireRec = record[1]
        recType = fireRec[0]      
        #recType = climateRec[0]
 
        if recType == "'cdata'": 
            station = int(climateRec[1])
            air_temp = int(climateRec[2])
            rel_humidity = float(climateRec[3])
            windspeed_knots = float(climateRec[4])
            maxwind_speed = float(climateRec[5])
            precipitation = climateRec[6]
            climateDf = pd.DataFrame({'Date':str(dt.datetime.now().strftime("%Y-%m-%d")), 'Station':station, 'Air_Temperature_Celcius':air_temp, 'Relative_Humidity':rel_humidity, 'WindSpeed_knots':windspeed_knots, 'Max_Wind_Speed':maxwind_speed, 'Precipitation ':precipitation}, index=[0])
            climateRecord = json.loads(climateDf.to_json(orient='records')) 
            print(climateRecord)
            climateCollection.insert_many(climateRecord)
        if recType == "'fdata'": 
            latitude = float(fireRec[1])
            longitude = float(fireRec[2])
            surfacetemp_kelvin = float(fireRec[3])
            power = float(fireRec[4])
            confidence = int(fireRec[5])
            surfacetemp_celcius = int(fireRec[6])
            fireDf = pd.DataFrame({'Date':str(dt.datetime.now().strftime("%Y-%m-%d")), 'Datetime':str(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")), 'Latitiude':latitude, 'Longitude':longitude, 'Surface_Temperature_Kelvin':surfacetemp_kelvin, 'Power':power, 'Confidence':confidence, 'Surface_Temperature_Celcius ':surfacetemp_celcius}, index=[0])
            #fireDf["Climate"] = climateRecord
            fireRecord = json.loads(fireDf.to_json(orient='records'))
            print(fireRecord)
            fireCollection.insert_many(fireRecord)        
    connection.close()   
    
#str(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
def sendPartition(iter):
    connection = MongoClient()
    db = connection.get_database('as2TaskB')
    climateCollection = db.climate
    fireCollection = db.fire
    
    for record in iter:
        datestamp = record[0]
        eventRec = record[1]
        eventType = eventRec[0]      
 
        if eventType == "'cdata'": 
            station = int(eventRec[1])
            air_temp = int(eventRec[2])
            rel_humidity = float(eventRec[3])
            windspeed_knots = float(eventRec[4])
            maxwind_speed = float(eventRec[5])
            precipitation = eventRec[6]
            climateDf = pd.DataFrame({'Date':str(dt.datetime.now().strftime("%Y-%m-%d")), 'Station':station, 'Air_Temperature_Celcius':air_temp, 'Relative_Humidity':rel_humidity, 'WindSpeed_knots':windspeed_knots, 'Max_Wind_Speed':maxwind_speed, 'Precipitation ':precipitation}, index=[0])
            climateRecord = json.loads(climateDf.to_json(orient='records')) 
            print(climateRecord)
            climateCollection.insert_many(climateRecord)
        if eventType == "'fdata'": 
            latitude = float(eventRec[1])
            longitude = float(eventRec[2])
            surfacetemp_kelvin = float(eventRec[3])
            power = float(eventRec[4])
            confidence = int(eventRec[5])
            surfacetemp_celcius = int(eventRec[6])
            fireDf = pd.DataFrame({'Date':str(dt.datetime.now().strftime("%Y-%m-%d")), 'Datetime':str(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")), 'Latitiude':latitude, 'Longitude':longitude, 'Surface_Temperature_Kelvin':surfacetemp_kelvin, 'Power':power, 'Confidence':confidence, 'Surface_Temperature_Celcius ':surfacetemp_celcius}, index=[0])
            #fireDf["Climate"] = climateRecord
            fireRecord = json.loads(fireDf.to_json(orient='records'))
            print(fireRecord)
            fireCollection.insert_many(fireRecord)        
    connection.close()   
    

In [ ]:
#S1 = ssc.socketTextStream("localhost", 9999)
#S2 = ssc.socketTextStream("localhost", 8085)

    # Create windowed stream
#    wS1 = S1.window(10)
#    wS2 = S2.window(1)

#    wS1.flatMap(lambda line: line.split(",")).pprint()
#    wS2.flatMap(lambda line: line.split(",")).pprint()

    # Perform join
#    joinedStream = wS1.join(wS2)

#    joinedStream.foreachRDD(lambda rdd: rdd.foreach(lambda x: print(x)))


#rdd = sc.parallelize([("red",20),("red",30),("blue", 100)])
#rdd2 = sc.parallelize([("red",40),("red",50),("yellow", 10000)])
#rdd.join(rdd2).collect()
# Gives [('red', (20, 40)), ('red', (20, 50)), ('red', (30, 40)), ('red', (30, 50))]

#("'2018-10-07 16:31:42'", ["'cdata'", '948701', '9', '43.2', '4.9', '8.0', "' 0.00I'", "'2018-10-07 16:31:42'"])



**Notes to examiner**

Below is an example of streaming from one of the Ports (Climate Data), as listening to both streams on both ports simulataneously doesn't seem to streaming the data. 

To listen to FireData stream, the climateData stream can be commented out, and the FireData stream uncommented.

Both the ClimateData and FireDatawork indepedently and write to MongoDb, however the joinedStream, required for the embedded data model in Task B, doesn't work

In [4]:
# We add this line to avoid an error : "Cannot run multiple SparkContexts at once". If there is an existing spark context, we will reuse it instead of creating a new context.
sc = SparkContext.getOrCreate()

# Create a local StreamingContext with as many working processors as possible and a batch interval of 10 seconds            
batch_interval = 5

# If there is no existing spark context, we now create a new context
if (sc is None):
    sc = SparkContext(master="local[3]", appName = "WordCountApp")
ssc = StreamingContext(sc, batch_interval)

host = "localhost"
port1 = "8085"
port2 = "9997"

climateStream = ssc.socketTextStream(host, int(port1))
fireStream = ssc.socketTextStream(host, int(port2))

#Create 5 second windows for each stream 
wclimateStream = climateStream.window(5)
wfireStream = fireStream.window(5)

#Map the windowed streams to (Datestamp, EventRecord) RDD's so they can be joined on datestamp.
climateRecs = wclimateStream.map(lambda climateRec: climateRec[1:-1].split(", ")).map(lambda climateRec: (climateRec[7][1:20], climateRec))
fireRecs = wfireStream.map(lambda fireRec: fireRec[1:-1].split(", ")).map(lambda fireRec: (fireRec[7][1:20], fireRec))

#create a joined stream to create RDDs of the form (Datestamp, (climateRec, fireRec))
joinedStream = climateRecs.join(fireRecs) #left outer join required as a climatic day may contain 0 or many fires.

#Store the event RDDs in the MongoDB database using sendPartition procedure
#climateRecs.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
#fireRecs.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
#joinedStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
joinedStream.foreachRDD(lambda rdd: rdd.foreach(lambda x: print(x)))

# Print the result                            
#climateRecs.pprint()
#fireRecs.pprint()
joinedStream.pprint()

#Start Streaming
ssc.start()
try:
    ssc.awaitTermination(timeout=60)
except KeyboardInterrupt:
    ssc.stop()
    sc.stop()


In [6]:
sc.stop()
ssc.stop()